In [1]:
import selenium 
import csv
import re
import pandas as pd

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from selenium.webdriver.chrome.options import Options

import time

In [2]:
sample_df = pd.read_csv("zero_results_second.csv")
# sample_df = sample_df.sample(200)

cas_df = sample_df[sample_df["CAS"] != "No CAS info"]
missing_df = sample_df[sample_df["CAS"] == "No CAS info"] 
sample_df

,Unnamed: 0,molecule,cas,best_match_url,results,query_url,query_type,Highest Development Event Status and Year,Highest Development Event Source ID,Highest Development Event Source URL,Earliest Approved Event Status and Year,Earliest Approved Event Source ID,Earliest Approved Event Source URL,conditions,phases,CAS
0,2,ALPHA-GALACTOSIDASE,7493-95-0,NaN,0,https://drugs.ncats.io/substances?q=(root_code...,ONLY CAS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7493-95-0
1,3,ALPHA AMYLASE,6401-81-6,NaN,0,https://drugs.ncats.io/substances?q=(root_code...,ONLY CAS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"9005-82-7, 6401-81-6, 1109-28-0"
2,11,ARALIA RACEMOSA,89957-50-6,NaN,0,https://drugs.ncats.io/substances?q=(root_code...,ONLY CAS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,89957-50-6
3,12,ARANEUS DIADEMATUS,91745-67-4,NaN,0,https://drugs.ncats.io/substances?q=(root_code...,ONLY CAS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,91745-67-4
4,14,AUROTHIOMALIC ACID,33796-26-8,NaN,0,https://drugs.ncats.io/substances?q=(root_code...,ONLY CAS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,33796-26-8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74,139,URTICA DIOICA,84012-40-8,NaN,0,https://drugs.ncats.io/substances?q=(root_code...,ONLY CAS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,84012-40-8
75,140,VALERIANA OFFICINALIS,81397-67-3,NaN,0,https://drugs.ncats.io/substances?q=(root_code...,ONLY CAS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,81397-67-3
76,141,VERBENA OFFICINALIS,977000-41-1,NaN,0,https://drugs.ncats.io/substances?q=(root_code...,ONLY CAS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,977000-41-1
77,142,VITAMIN F,11006-87-4,NaN,0,https://drugs.ncats.io/substances?q=(root_code...,ONLY CAS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"7771-44-0, 11006-87-4, 506-32-1"


In [3]:
cas_df = cas_df[['molecule', 'CAS']]

In [4]:
cas_df.head(50)

,molecule,CAS
0,ALPHA-GALACTOSIDASE,7493-95-0
1,ALPHA AMYLASE,"9005-82-7, 6401-81-6, 1109-28-0"
2,ARALIA RACEMOSA,89957-50-6
3,ARANEUS DIADEMATUS,91745-67-4
4,AUROTHIOMALIC ACID,33796-26-8
5,BERBERIS VULGARIS,84649-92-3
6,BRASSICA NAPUS,173740-48-2
7,CADEXOMER IODINE,94820-09-4
8,CALAMINE,"8011-96-9, 12063-19-3"
9,CAPSICUM,"84625-29-6, 85940-30-3, 977007-72-9"


In [5]:
def get_third_cas(cas_string):
    # Replace all semicolons with commas
    if len(cas_string.split(',')) > 2:
        return cas_string.split(',')[2].strip()
    else:
        return cas_string.split(',')[0].strip()

In [6]:
# Apply this function to the entire CAS column
cas_df['CAS'] = cas_df['CAS'].apply(get_third_cas)

molecules = list(zip(cas_df['molecule'], cas_df['CAS']))
molecules

# missing_df['CAS'] = missing_df['CAS'].apply(get_first_cas)
# missing_arr = missing_df['CAS'].tolist()

[('ALPHA-GALACTOSIDASE', '7493-95-0'),
 ('ALPHA AMYLASE', '1109-28-0'),
 ('ARALIA RACEMOSA', '89957-50-6'),
 ('ARANEUS DIADEMATUS', '91745-67-4'),
 ('AUROTHIOMALIC ACID', '33796-26-8'),
 ('BERBERIS VULGARIS', '84649-92-3'),
 ('BRASSICA NAPUS', '173740-48-2'),
 ('CADEXOMER IODINE', '94820-09-4'),
 ('CALAMINE', '8011-96-9'),
 ('CAPSICUM', '977007-72-9'),
 ('CITRUS DECUMANA', '90045-43-5'),
 ('DACTYLIS GLOMERATA', '94349-98-1'),
 ('DEXTRIN', '9004-53-9'),
 ('DOLOMITE', '7000-29-5'),
 ('DROSERA ROTUNDIFOLIA', '84696-10-6'),
 ('ECHINACEA ANGUSTIFOLIA', '84696-11-7'),
 ('ECHINACEA PURPUREA', '90028-20-9'),
 ('ELETTARIA CARDAMOMUM', '977005-95-0'),
 ('EUPATORIUM PERFOLIATUM', '84625-35-4'),
 ('FACTOR VIII INHIBITOR BYPASSING FRACTION', '78690-39-8'),
 ('FACTOR XIII', '97089-56-0'),
 ('FUCUS VESICULOSUS', '977001-75-4'),
 ('FURAZOLIDONE', '67-45-8'),
 ('GARCINIA MANGOSTANA', '107390-08-9'),
 ('GAULTHERIA PROCUMBENS', '977092-74-2'),
 ('GELSEMIUM SEMPERVIRENS', '84650-08-8'),
 ('GERANIUM MACULA

In [7]:
import urllib

def setup_webdriver():
    '''Initializes a headless selenium webdriver'''
    chrome_options = Options()
    chrome_options.add_argument("--headless")
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=chrome_options)
    return driver

def cas_url_inxight(molecule):
    cas = molecule[1]
    cas_formatted = cas.replace('-', '%20')
    return f'https://drugs.ncats.io/substances?q=(root_codes_CAS:\"{cas_formatted}\")'

def name_url_inxight(molecule):
    '''Generates an Inxight URL for a given CAS'''
    name = molecule[0]
    cas = molecule[1]
    cas_formatted = cas.replace('-', '%20')
    name_encoded = urllib.parse.quote(f'^{name}')
    return f'https://drugs.ncats.io/substances?q=(root_codes_CAS:\"{cas_formatted}\")%20AND%20(root_names_name:\"{name_encoded}\")'

def name_exact_inxight(molecule):
    '''Generates an Inxight URL for a given CAS'''
    name = molecule[0]
    cas = molecule[1]
    cas_formatted = cas.replace('-', '%20')
    name_encoded = urllib.parse.quote(f'^{name}')
    return f'https://drugs.ncats.io/substances?q=(root_codes_CAS:\"{cas_formatted}\")%20AND%20(root_names_name:\"{name_encoded}$\")'

In [8]:
def get_inxight_url(molecule, driver):
    '''
    Parameters
    ----------
    A molecule's CAS and an initialised webdriver.

    Returns
    -------
    The top Inxight search result for a given CAS number
    '''
    
    url = "N/A"  # Default in case of failure
    query = cas_url_inxight(molecule)
    query_type = "ONLY CAS"
    driver.get(query)
    
    try:
        WebDriverWait(driver, 3).until(EC.visibility_of_element_located((By.CSS_SELECTOR, 'span#record-count:nth-child(2)')))
        elements = driver.find_elements(By.CSS_SELECTOR, 'span#record-count:nth-child(2)')
        if elements:
            count = int(elements[0].text)
            print(count)
        else:
            print("Element not found.")
            
    except TimeoutException:
        print("Element not found within specified time.")
        count = 0

    if count > 1:
        query = name_exact_inxight(molecule)
        query_type = "EXACT NAME"
        driver.get(query)
    else:
        pass
        
    try:
        WebDriverWait(driver, 3).until(EC.visibility_of_element_located((By.CSS_SELECTOR, 'a[id="card-title"]')))
        elements = driver.find_elements(By.CSS_SELECTOR, 'a[id="card-title"]')
        if elements:
            element = elements[0]
            url = element.get_attribute('href')

    except (NoSuchElementException, TimeoutException):
        query = name_url_inxight(molecule)
        query_type = "APPROXIMATE NAME"
        driver.get(query)
        
        try:
            WebDriverWait(driver, 3).until(EC.visibility_of_element_located((By.CSS_SELECTOR, 'a[id="card-title"]')))
            elements = driver.find_elements(By.CSS_SELECTOR, 'a[id="card-title"]')
            if elements:
                element = elements[0]
                url = element.get_attribute('href')

        except (NoSuchElementException, TimeoutException):
            query = cas_url_inxight(molecule)
            query_type = "ONLY CAS"
            driver.get(query)

            try:
                WebDriverWait(driver, 3).until(EC.visibility_of_element_located((By.CSS_SELECTOR, 'a[id="card-title"]')))
                elements = driver.find_elements(By.CSS_SELECTOR, 'a[id="card-title"]')
                if elements:
                    element = elements[0]
                    url = element.get_attribute('href')
            
            except (NoSuchElementException, TimeoutException):
                pass
            
    return molecule, url, count, query, query_type

def get_best_urls(mol_arr):
    '''
    Parameters
    ----------
    An array of CAS values

    Returns
    -------
    An array of associated top matches
    '''
    
    best_matches = []
    driver = setup_webdriver()  
    try:
        for molecule in mol_arr:
            molecule, url, count, query, query_type = get_inxight_url(molecule, driver)
            best_matches.append((molecule, url, count, query, query_type))
    finally:
        driver.quit()  

    return best_matches

In [9]:
import requests

def get_additional_data(id):
    r = requests.get(f'https://drugs.ncats.io/api/v1/substances({id})/@additional')
    if 200 == r.status_code:
        return r.json()
    return None

In [10]:
def extract_conditions_and_phases(data):
    conditions_list = []
    highest_approval_list = []

    if data is None:
        return conditions_list, highest_approval_list

    for entry in data:
        if entry['name'] == 'Conditions' and 'value' in entry:
            condition_info = entry['value']
            
            # Extract the condition name
            label = condition_info.get('label')
            if label:
                conditions_list.append(label)
            
            # Extract the highest phase of approval
            highest_phase = condition_info.get('highestPhase')
            if highest_phase:
                highest_approval_list.append(highest_phase)

    return conditions_list, highest_approval_list

In [11]:
# def extract_event_details(data):
#     event_details = {}
    
#     for item in data:
#         if 'value' in item and isinstance(item['value'], dict):  # Ensure 'value' is a dictionary
#             details = item['value']
#             if 'status' in details and 'sourceID' in details:
#                 if item['name'] == 'Highest Development Event' or item['name'] == 'Earliest Approved Event':
#                     # Gather additional details
#                     source_id = details.get('sourceID', 'No Source ID')
#                     source_url = details.get('sourceURL', 'No Source URL')
                    
#                     detail_info = {
#                         'Status and Year': f"{details['status']} {details.get('year', '')}",
#                         'Source ID': source_id,
#                         'Source URL': source_url
#                     }
                    
#                     # Use the 'name' of the event as the key in the dictionary
#                     event_details[item['name']] = detail_info
    
#     return event_details

def extract_event_details(data):
    event_details = {}
    if data is None:  # Check if data is None before iterating
        return event_details

    for item in data:
        if 'value' in item and isinstance(item['value'], dict):
            details = item['value']
            if 'status' in details and 'sourceID' in details:
                if item['name'] in ['Highest Development Event', 'Earliest Approved Event']:
                    detail_info = {
                        'Status and Year': f"{details['status']} {details.get('year', '')}",
                        'Source ID': details.get('sourceID', 'No Source ID'),
                        'Source URL': details.get('sourceURL', 'No Source URL')
                    }
                    event_details[item['name']] = detail_info
    return event_details

In [12]:
links = get_best_urls(molecules)
links

Element not found within specified time.
1
Element not found within specified time.
Element not found within specified time.
Element not found within specified time.
Element not found within specified time.
Element not found within specified time.
Element not found within specified time.
Element not found within specified time.
Element not found within specified time.
Element not found within specified time.
Element not found within specified time.
Element not found within specified time.
1
Element not found within specified time.
Element not found within specified time.
Element not found within specified time.
Element not found within specified time.
Element not found within specified time.
Element not found within specified time.
Element not found within specified time.
Element not found within specified time.
1
Element not found within specified time.
Element not found within specified time.
Element not found within specified time.
Element not found within specified time.
Element no

[(('ALPHA-GALACTOSIDASE', '7493-95-0'),
  'N/A',
  0,
  'https://drugs.ncats.io/substances?q=(root_codes_CAS:"7493%2095%200")',
  'ONLY CAS'),
 (('ALPHA AMYLASE', '1109-28-0'),
  'https://drugs.ncats.io/drug/639K0T34IK',
  1,
  'https://drugs.ncats.io/substances?q=(root_codes_CAS:"1109%2028%200")',
  'ONLY CAS'),
 (('ARALIA RACEMOSA', '89957-50-6'),
  'N/A',
  0,
  'https://drugs.ncats.io/substances?q=(root_codes_CAS:"89957%2050%206")',
  'ONLY CAS'),
 (('ARANEUS DIADEMATUS', '91745-67-4'),
  'N/A',
  0,
  'https://drugs.ncats.io/substances?q=(root_codes_CAS:"91745%2067%204")',
  'ONLY CAS'),
 (('AUROTHIOMALIC ACID', '33796-26-8'),
  'N/A',
  0,
  'https://drugs.ncats.io/substances?q=(root_codes_CAS:"33796%2026%208")',
  'ONLY CAS'),
 (('BERBERIS VULGARIS', '84649-92-3'),
  'N/A',
  0,
  'https://drugs.ncats.io/substances?q=(root_codes_CAS:"84649%2092%203")',
  'ONLY CAS'),
 (('BRASSICA NAPUS', '173740-48-2'),
  'N/A',
  0,
  'https://drugs.ncats.io/substances?q=(root_codes_CAS:"173740

In [13]:
# new_links_df = pd.DataFrame(links, columns=['molecule', 'query_url', 'results', 'best_match_url', ''])
# new_links_df.head(2)
# links = new_links_df['best_match_url'].to_list()
# links
# identifiers = [url.split('/')[-1] for _, url in links]
# identifiers

data = []
for link in links:
    molecule_name, cas_number = link[0]
    drug_url, value, substance_url, query_type = link[1:]
    data.append([molecule_name, cas_number, drug_url, value, substance_url, query_type])
    
new_links_df = pd.DataFrame(data, columns=['molecule', 'cas', 'best_match_url', 'results', 'query_url', 'query_type'])
new_links_df.head(2)

,molecule,cas,best_match_url,results,query_url,query_type
0,ALPHA-GALACTOSIDASE,7493-95-0,N/A,0,https://drugs.ncats.io/substances?q=(root_code...,ONLY CAS
1,ALPHA AMYLASE,1109-28-0,https://drugs.ncats.io/drug/639K0T34IK,1,https://drugs.ncats.io/substances?q=(root_code...,ONLY CAS


In [14]:
identifier_arr = new_links_df['best_match_url'].to_list()
identifier_arr
identifiers = [url.split('/')[-1] for url in identifier_arr]

In [15]:
identifiers = [identifier if identifier != 'A' else 'MISSING' for identifier in identifiers]

In [16]:
len(identifiers)

79

In [17]:
def extract_events(identifiers):
    data = []
    events = []

    for identifier in identifiers:
        data = get_additional_data(identifier)
        print(extract_event_details(data))
        events.append(extract_event_details(data))

    return events
        
events = extract_events(identifiers)

# def extract_events(identifiers):
#     events = []
#     for identifier in identifiers:
#         data = get_additional_data(identifier)
#         if data is not None:  # Check if data is None
#             event_details = extract_event_details(data)
#             events.append(event_details)
#         else:
#             print(f"No data available for identifier {identifier}")  # Or handle it differently
#     return events

{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{'Highest Development Event': {'Status and Year': 'Clinical 2013', 'Source ID': 'NCT01831492: Not Applicable Interventional Completed Acidosis', 'Source URL': 'https://clinicaltrials.gov/ct2/show/NCT01831492'}}
{}
{}
{}
{}
{}
{}
{}
{}
{'Highest Development Event': {'Status and Year': 'US Previously Marketed 1961', 'Source ID': 'FUROXONE by SHIRE', 'Source URL': 'https://www.accessdata.fda.gov/scripts/cder/daf/index.cfm?event=overview.process&ApplNo=011270'}, 'Earliest Approved Event': {'Status and Year': 'US Previously Marketed 1957', 'Source ID': 'Tricofuron by Eaton', 'Source URL': 'OB NME Appendix 1950-1985'}}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{'Highest Development Event': {'Status and Year': 'US Approved Rx 1985', 'Source ID': 'NDA018677', 'Source URL': 'https://www.accessdata.fda.gov/scripts/cder/daf/index.cfm?event=overview.process&ApplNo=018677'}, 'Earliest Approved Event': {'Status and Year': 'US Approved

In [18]:
def extract_conditions(identifiers):
    conditions = []

    for identifier in identifiers:
        data = get_additional_data(identifier)
        conditions.append(extract_conditions_and_phases(data))

    return conditions
        
conditions_arr = extract_conditions(identifiers)

In [19]:
events

[{},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {'Highest Development Event': {'Status and Year': 'Clinical 2013',
   'Source ID': 'NCT01831492: Not Applicable Interventional Completed Acidosis',
   'Source URL': 'https://clinicaltrials.gov/ct2/show/NCT01831492'}},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {'Highest Development Event': {'Status and Year': 'US Previously Marketed 1961',
   'Source ID': 'FUROXONE by SHIRE',
   'Source URL': 'https://www.accessdata.fda.gov/scripts/cder/daf/index.cfm?event=overview.process&ApplNo=011270'},
  'Earliest Approved Event': {'Status and Year': 'US Previously Marketed 1957',
   'Source ID': 'Tricofuron by Eaton',
   'Source URL': 'OB NME Appendix 1950-1985'}},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {'Highest Development Event': {'Status and Year': 'US Approved Rx 1985',
   'Source ID': 'NDA018677',
   'Source URL': 'https://www.accessdata.fda.gov/scripts

In [20]:
conditions_arr

[([], []),
 ([], []),
 ([], []),
 ([], []),
 ([], []),
 ([], []),
 ([], []),
 ([], []),
 ([], []),
 ([], []),
 ([], []),
 ([], []),
 ([], []),
 (['Unknown'], []),
 ([], []),
 ([], []),
 ([], []),
 ([], []),
 ([], []),
 ([], []),
 ([], []),
 ([], []),
 (['Bacterial or protozoal enteritis', 'Bacterial or protozoal diarrhea'],
  ['Approved', 'Approved']),
 ([], []),
 ([], []),
 ([], []),
 ([], []),
 ([], []),
 ([], []),
 ([], []),
 ([], []),
 ([], []),
 ([], []),
 ([], []),
 ([], []),
 ([], []),
 ([], []),
 ([], []),
 ([], []),
 ([], []),
 ([], []),
 ([], []),
 ([], []),
 ([], []),
 ([], []),
 ([], []),
 (['Nausea and vomiting'], ['Approved']),
 ([], []),
 ([], []),
 ([], []),
 ([], []),
 ([], []),
 ([], []),
 ([], []),
 ([], []),
 ([], []),
 ([], []),
 ([], []),
 ([], []),
 ([], []),
 ([], []),
 ([], []),
 ([], []),
 ([], []),
 ([], []),
 ([], []),
 ([], []),
 ([], []),
 ([], []),
 ([], []),
 ([], []),
 ([], []),
 (['Unknown', 'Ovarian cancer', 'Breast cancer'], ['Phase III', 'Approved']

In [21]:
data = []
for event in events:
    record = {}
    for key, value in event.items():
        for sub_key, sub_value in value.items():
            record[f"{key} {sub_key}"] = sub_value
    data.append(record)

df = pd.DataFrame(data)

In [22]:
df

,Highest Development Event Status and Year,Highest Development Event Source ID,Highest Development Event Source URL,Earliest Approved Event Status and Year,Earliest Approved Event Source ID,Earliest Approved Event Source URL
0,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
74,NaN,NaN,NaN,NaN,NaN,NaN
75,NaN,NaN,NaN,NaN,NaN,NaN
76,NaN,NaN,NaN,NaN,NaN,NaN
77,Possibly Marketed Outside US 2013,LipoGel CR Base by Southeastern Medical Techno...,https://dailymed.nlm.nih.gov/dailymed/drugInfo...,Possibly Marketed Outside US 2013,LipoGel CR Base by Southeastern Medical Techno...,https://dailymed.nlm.nih.gov/dailymed/drugInfo...


In [23]:
rows = [{'conditions': conditions, 'phases': phases} for conditions, phases in conditions_arr]

# Create DataFrame
conditions_df = pd.DataFrame(rows)
conditions_df['conditions'] = conditions_df['conditions'].apply(lambda x: '; '.join(x))
conditions_df['phases'] = conditions_df['phases'].apply(lambda x: '; '.join(x))
conditions_df

,conditions,phases
0,,
1,,
2,,
3,,
4,,
...,...,...
74,,
75,,
76,,
77,,


In [24]:
merged = pd.concat([df, conditions_df], axis=1)
merged

,Highest Development Event Status and Year,Highest Development Event Source ID,Highest Development Event Source URL,Earliest Approved Event Status and Year,Earliest Approved Event Source ID,Earliest Approved Event Source URL,conditions,phases
0,NaN,NaN,NaN,NaN,NaN,NaN,,
1,NaN,NaN,NaN,NaN,NaN,NaN,,
2,NaN,NaN,NaN,NaN,NaN,NaN,,
3,NaN,NaN,NaN,NaN,NaN,NaN,,
4,NaN,NaN,NaN,NaN,NaN,NaN,,
...,...,...,...,...,...,...,...,...
74,NaN,NaN,NaN,NaN,NaN,NaN,,
75,NaN,NaN,NaN,NaN,NaN,NaN,,
76,NaN,NaN,NaN,NaN,NaN,NaN,,
77,Possibly Marketed Outside US 2013,LipoGel CR Base by Southeastern Medical Techno...,https://dailymed.nlm.nih.gov/dailymed/drugInfo...,Possibly Marketed Outside US 2013,LipoGel CR Base by Southeastern Medical Techno...,https://dailymed.nlm.nih.gov/dailymed/drugInfo...,,


In [25]:
new_links_df

,molecule,cas,best_match_url,results,query_url,query_type
0,ALPHA-GALACTOSIDASE,7493-95-0,N/A,0,https://drugs.ncats.io/substances?q=(root_code...,ONLY CAS
1,ALPHA AMYLASE,1109-28-0,https://drugs.ncats.io/drug/639K0T34IK,1,https://drugs.ncats.io/substances?q=(root_code...,ONLY CAS
2,ARALIA RACEMOSA,89957-50-6,N/A,0,https://drugs.ncats.io/substances?q=(root_code...,ONLY CAS
3,ARANEUS DIADEMATUS,91745-67-4,N/A,0,https://drugs.ncats.io/substances?q=(root_code...,ONLY CAS
4,AUROTHIOMALIC ACID,33796-26-8,N/A,0,https://drugs.ncats.io/substances?q=(root_code...,ONLY CAS
...,...,...,...,...,...,...
74,URTICA DIOICA,84012-40-8,N/A,0,https://drugs.ncats.io/substances?q=(root_code...,ONLY CAS
75,VALERIANA OFFICINALIS,81397-67-3,N/A,0,https://drugs.ncats.io/substances?q=(root_code...,ONLY CAS
76,VERBENA OFFICINALIS,977000-41-1,N/A,0,https://drugs.ncats.io/substances?q=(root_code...,ONLY CAS
77,VITAMIN F,506-32-1,https://drugs.ncats.io/drug/27YG812J1I,1,https://drugs.ncats.io/substances?q=(root_code...,ONLY CAS


In [26]:
result = pd.concat([new_links_df, merged], axis=1)
result

,molecule,cas,best_match_url,results,query_url,query_type,Highest Development Event Status and Year,Highest Development Event Source ID,Highest Development Event Source URL,Earliest Approved Event Status and Year,Earliest Approved Event Source ID,Earliest Approved Event Source URL,conditions,phases
0,ALPHA-GALACTOSIDASE,7493-95-0,N/A,0,https://drugs.ncats.io/substances?q=(root_code...,ONLY CAS,NaN,NaN,NaN,NaN,NaN,NaN,,
1,ALPHA AMYLASE,1109-28-0,https://drugs.ncats.io/drug/639K0T34IK,1,https://drugs.ncats.io/substances?q=(root_code...,ONLY CAS,NaN,NaN,NaN,NaN,NaN,NaN,,
2,ARALIA RACEMOSA,89957-50-6,N/A,0,https://drugs.ncats.io/substances?q=(root_code...,ONLY CAS,NaN,NaN,NaN,NaN,NaN,NaN,,
3,ARANEUS DIADEMATUS,91745-67-4,N/A,0,https://drugs.ncats.io/substances?q=(root_code...,ONLY CAS,NaN,NaN,NaN,NaN,NaN,NaN,,
4,AUROTHIOMALIC ACID,33796-26-8,N/A,0,https://drugs.ncats.io/substances?q=(root_code...,ONLY CAS,NaN,NaN,NaN,NaN,NaN,NaN,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74,URTICA DIOICA,84012-40-8,N/A,0,https://drugs.ncats.io/substances?q=(root_code...,ONLY CAS,NaN,NaN,NaN,NaN,NaN,NaN,,
75,VALERIANA OFFICINALIS,81397-67-3,N/A,0,https://drugs.ncats.io/substances?q=(root_code...,ONLY CAS,NaN,NaN,NaN,NaN,NaN,NaN,,
76,VERBENA OFFICINALIS,977000-41-1,N/A,0,https://drugs.ncats.io/substances?q=(root_code...,ONLY CAS,NaN,NaN,NaN,NaN,NaN,NaN,,
77,VITAMIN F,506-32-1,https://drugs.ncats.io/drug/27YG812J1I,1,https://drugs.ncats.io/substances?q=(root_code...,ONLY CAS,Possibly Marketed Outside US 2013,LipoGel CR Base by Southeastern Medical Techno...,https://dailymed.nlm.nih.gov/dailymed/drugInfo...,Possibly Marketed Outside US 2013,LipoGel CR Base by Southeastern Medical Techno...,https://dailymed.nlm.nih.gov/dailymed/drugInfo...,,


In [27]:
zeros_df = result[result['results'] == 0]
zeros_df

,molecule,cas,best_match_url,results,query_url,query_type,Highest Development Event Status and Year,Highest Development Event Source ID,Highest Development Event Source URL,Earliest Approved Event Status and Year,Earliest Approved Event Source ID,Earliest Approved Event Source URL,conditions,phases
0,ALPHA-GALACTOSIDASE,7493-95-0,N/A,0,https://drugs.ncats.io/substances?q=(root_code...,ONLY CAS,NaN,NaN,NaN,NaN,NaN,NaN,,
2,ARALIA RACEMOSA,89957-50-6,N/A,0,https://drugs.ncats.io/substances?q=(root_code...,ONLY CAS,NaN,NaN,NaN,NaN,NaN,NaN,,
3,ARANEUS DIADEMATUS,91745-67-4,N/A,0,https://drugs.ncats.io/substances?q=(root_code...,ONLY CAS,NaN,NaN,NaN,NaN,NaN,NaN,,
4,AUROTHIOMALIC ACID,33796-26-8,N/A,0,https://drugs.ncats.io/substances?q=(root_code...,ONLY CAS,NaN,NaN,NaN,NaN,NaN,NaN,,
5,BERBERIS VULGARIS,84649-92-3,N/A,0,https://drugs.ncats.io/substances?q=(root_code...,ONLY CAS,NaN,NaN,NaN,NaN,NaN,NaN,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73,TRIBULUS TERRESTRIS,90131-68-3,N/A,0,https://drugs.ncats.io/substances?q=(root_code...,ONLY CAS,NaN,NaN,NaN,NaN,NaN,NaN,,
74,URTICA DIOICA,84012-40-8,N/A,0,https://drugs.ncats.io/substances?q=(root_code...,ONLY CAS,NaN,NaN,NaN,NaN,NaN,NaN,,
75,VALERIANA OFFICINALIS,81397-67-3,N/A,0,https://drugs.ncats.io/substances?q=(root_code...,ONLY CAS,NaN,NaN,NaN,NaN,NaN,NaN,,
76,VERBENA OFFICINALIS,977000-41-1,N/A,0,https://drugs.ncats.io/substances?q=(root_code...,ONLY CAS,NaN,NaN,NaN,NaN,NaN,NaN,,


In [28]:
found = result[result['results'] != 0]

In [29]:
found

,molecule,cas,best_match_url,results,query_url,query_type,Highest Development Event Status and Year,Highest Development Event Source ID,Highest Development Event Source URL,Earliest Approved Event Status and Year,Earliest Approved Event Source ID,Earliest Approved Event Source URL,conditions,phases
1,ALPHA AMYLASE,1109-28-0,https://drugs.ncats.io/drug/639K0T34IK,1,https://drugs.ncats.io/substances?q=(root_code...,ONLY CAS,NaN,NaN,NaN,NaN,NaN,NaN,,
13,DOLOMITE,7000-29-5,https://drugs.ncats.io/drug/UG9H2002BF,1,https://drugs.ncats.io/substances?q=(root_code...,ONLY CAS,Clinical 2013,NCT01831492: Not Applicable Interventional Com...,https://clinicaltrials.gov/ct2/show/NCT01831492,NaN,NaN,NaN,Unknown,
22,FURAZOLIDONE,67-45-8,https://drugs.ncats.io/drug/5J9CPU3RE0,1,https://drugs.ncats.io/substances?q=(root_code...,ONLY CAS,US Previously Marketed 1961,FUROXONE by SHIRE,https://www.accessdata.fda.gov/scripts/cder/da...,US Previously Marketed 1957,Tricofuron by Eaton,OB NME Appendix 1950-1985,Bacterial or protozoal enteritis; Bacterial or...,Approved; Approved
46,NABILONE,51022-71-0,https://drugs.ncats.io/drug/2N4O9L084N,1,https://drugs.ncats.io/substances?q=(root_code...,ONLY CAS,US Approved Rx 1985,NDA018677,https://www.accessdata.fda.gov/scripts/cder/da...,US Approved Rx 1985,NDA018677,https://www.accessdata.fda.gov/scripts/cder/da...,Nausea and vomiting,Approved
72,TALAZOPARIB,1373431-65-2,https://drugs.ncats.io/drug/02WK9U5NZC,1,https://drugs.ncats.io/substances?q=(root_code...,ONLY CAS,US Approved Rx 2018,NDA211651,https://www.accessdata.fda.gov/scripts/cder/da...,US Approved Rx 2018,NDA211651,https://www.accessdata.fda.gov/scripts/cder/da...,Unknown; Ovarian cancer; Breast cancer,Phase III; Approved
77,VITAMIN F,506-32-1,https://drugs.ncats.io/drug/27YG812J1I,1,https://drugs.ncats.io/substances?q=(root_code...,ONLY CAS,Possibly Marketed Outside US 2013,LipoGel CR Base by Southeastern Medical Techno...,https://dailymed.nlm.nih.gov/dailymed/drugInfo...,Possibly Marketed Outside US 2013,LipoGel CR Base by Southeastern Medical Techno...,https://dailymed.nlm.nih.gov/dailymed/drugInfo...,,


In [30]:
zeros_df.to_csv("zero_results_third.csv")
found.to_csv("found_results_third.csv", index = False)

In [31]:
found.to_csv("found_results_third_real.csv", index = False)